In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')
train_paths += glob('../feature/org_use/526*_train.gz')
test_paths  += glob('../feature/org_use/526*_test.gz')
train_paths += glob('../feature/raw_use/ker__uid*_train.gz')
test_paths  += glob('../feature/raw_use/ker__uid*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('fill__M')
#                or path.count('V')
#                or path.count('C')
               or path.count('D1_')
#                or (path.count('526') and path.count('mean'))
#                or path.count('uid_')
               or path.count('uid2_t')
               or path.count('uid3_t')
               or path.count('uid4_t')
#                or path.count('uid5_t')
#                or path.count('card')
#                or path.count('addr')
               or path.count('domain')
               or path.count('TransactionAmt')
               or path.count('Product')
               or path.count('hour')
               or path.count('zone')
               or path.count('500')
               or path.count('510')
               )
#                and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
               and not path.count('130')
#                or path.count('500')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('fill__M')
#                or path.count('V')
#                or path.count('C')
               or path.count('D1_')
#                or (path.count('526') and path.count('mean'))
#                or path.count('uid_')
               or path.count('uid2_t')
               or path.count('uid3_t')
               or path.count('uid4_t')
#                or path.count('uid5_t')
#                or path.count('card')
#                or path.count('addr')
               or path.count('domain')
               or path.count('TransactionAmt')
               or path.count('Product')
               or path.count('hour')
               or path.count('zone')
               or path.count('500')
               or path.count('510')
               )
#                and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
               and not path.count('130')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]

Process ForkPoolWorker-185:
Process ForkPoolWorker-72:
Process ForkPoolWorker-93:
Process ForkPoolWorker-27:
Process ForkPoolWorker-66:
Process ForkPoolWorker-48:
Process ForkPoolWorker-90:
Process ForkPoolWorker-70:
Process ForkPoolWorker-9:
Process ForkPoolWorker-89:
Process ForkPoolWorker-123:
Process ForkPoolWorker-86:
Process ForkPoolWorker-175:
Process ForkPoolWorker-161:
Process ForkPoolWorker-155:
Process ForkPoolWorker-17:
Process ForkPoolWorker-139:
Process ForkPoolWorker-173:
Process ForkPoolWorker-167:
Process ForkPoolWorker-172:
Process ForkPoolWorker-122:
Process ForkPoolWorker-47:
Process ForkPoolWorker-131:
Process ForkPoolWorker-54:
Process ForkPoolWorker-67:
Process ForkPoolWorker-109:
Process ForkPoolWorker-107:
Process ForkPoolWorker-177:
Process ForkPoolWorker-148:
Process ForkPoolWorker-30:
Process ForkPoolWorker-61:
Process ForkPoolWorker-80:
Process ForkPoolWorker-49:
Process ForkPoolWorker-158:
Process ForkPoolWorker-160:
Process ForkPoolWorker-88:
Process Fork

In [3]:
START_DATE = '2017-12-01'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
data['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
data['date'] = data['datetime'].map(lambda x: x.date())

list_regist = []
for d, diff in tqdm(data[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

data['Regist_date'] = list_regist

100%|██████████| 1097231/1097231 [00:07<00:00, 152954.36it/s]


In [13]:
from itertools import combinations, product

cols_card = [col for col in data.columns if col.count('card') and col.count('fill_')]
cols_addr = [col for col in data.columns if col.count('addr1') and col.count('fill')]
cols_uid = [col for col in data.columns if col.count('uid') and not col.count('Amt')]
cols_M = [col for col in data.columns if col.count('M')]
cols_domain = [col for col in data.columns if col.count('domain') and col.count('fill')]
cols_501 = [col for col in data.columns if col.count('501')]
cols_500 = [col for col in data.columns if col.count('500')]
cols_time = ['hour', 'time_zone']
cos_pd = ['ProductCD']
cols_card_addr = list(product(cols_card, ['addr1']))

In [7]:
list_base_key = cols_uid
col_cat = 'ProductCD'
col_cat = 'TransactionAmt'
dir_save = 'valid_use'

for base_key in list_base_key:

    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
    
#     base_key += ['Regist_date']
    base_key += ['Regist_date', 'ProductCD']
    name_base = '-'.join(base_key)
    
    base = data[base_key + [col_cat]]
    
    all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
        f'count': 'count'
    })
    
    part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
        f'count_cat': 'count'
    })
    
    tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)
    
    tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
    
    tmp_max = tmp.groupby(base_key)['cat_ratio'].max().to_frame('cat_ratio_max')
    tmp = tmp.merge(tmp_max, how='inner', on=base_key)
    tmp['cat_ratio_main'] = tmp['cat_ratio'] / tmp['cat_ratio_max']
    
    tmp.drop(['count', 'count_cat'], axis=1, inplace=True)
    
#     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
#     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
#     result.columns = cols_save

    df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
    fname = f"main_cnt_ratio-{col_cat}-{name_base}"
    df_feat.rename(columns={'cat_ratio_main': fname}, inplace=True)
    cols_save = [fname]
    
    base_train = df_feat.iloc[:df_train.shape[0]]
    base_test  = df_feat.iloc[df_train.shape[0]:]
    
    save_feature(base_train[cols_save], '536', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(base_test[cols_save],  '536', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | main_cnt_ratio-TransactionAmt-ker__uid2-Regist_date-ProductCD
(506691,) | main_cnt_ratio-TransactionAmt-ker__uid2-Regist_date-ProductCD
(590540,) | main_cnt_ratio-TransactionAmt-ker__uid3-Regist_date-ProductCD
(506691,) | main_cnt_ratio-TransactionAmt-ker__uid3-Regist_date-ProductCD
(590540,) | main_cnt_ratio-TransactionAmt-ker__uid3_TransactionAmt_std-Regist_date-ProductCD
(506691,) | main_cnt_ratio-TransactionAmt-ker__uid3_TransactionAmt_std-Regist_date-ProductCD
(590540,) | main_cnt_ratio-TransactionAmt-ker__uid4-Regist_date-ProductCD
(506691,) | main_cnt_ratio-TransactionAmt-ker__uid4-Regist_date-ProductCD
(590540,) | main_cnt_ratio-TransactionAmt-ker__uid4_TransactionAmt_std-Regist_date-ProductCD
(506691,) | main_cnt_ratio-TransactionAmt-ker__uid4_TransactionAmt_std-Regist_date-ProductCD
(590540,) | main_cnt_ratio-TransactionAmt-ker__uid5_TransactionAmt_mean-Regist_date-ProductCD
(506691,) | main_cnt_ratio-TransactionAmt-ker__uid5_TransactionAmt_mean-Regist_date-Produc

In [15]:
list_base_key = cols_uid[2:]
# list_base_key = cols_card_addr
cat_list = cols_M + cols_domain + cols_501 + cols_time + cols_500
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    if str(type(base_key)).count('tuple'):
        base_key = list(base_key)
    else:
        base_key = [base_key]
        
#     base_key += ['Regist_date']
    base_key += ['Regist_date', 'ProductCD']
    name_base = '-'.join(base_key)
    
    for col_cat in tqdm(cat_list):

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)
        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        
        tmp_max = tmp.groupby(base_key)['cat_ratio'].max().to_frame('cat_ratio_max')
        tmp = tmp.merge(tmp_max, how='inner', on=base_key)
        tmp['cat_ratio_main'] = tmp['cat_ratio'] / tmp['cat_ratio_max']

        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"main_cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio_main': fname}, inplace=True)
        cols_save = [fname]
        
        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)



  0%|          | 0/152 [00:00<?, ?it/s]

(590540,) | main_cnt_ratio-fill__M1-ker__uid4-Regist_date-ProductCD
(506691,) | main_cnt_ratio-fill__M1-ker__uid4-Regist_date-ProductCD




  1%|          | 1/152 [00:07<17:49,  7.09s/it]

(590540,) | main_cnt_ratio-fill__M2-ker__uid4-Regist_date-ProductCD
(506691,) | main_cnt_ratio-fill__M2-ker__uid4-Regist_date-ProductCD




  1%|▏         | 2/152 [00:12<16:39,  6.66s/it]

...



 99%|█████████▊| 150/152 [13:08<00:07,  3.53s/it]

(590540,) | main_cnt_ratio-500__screen_width-ker__uid4-Regist_date-ProductCD
(506691,) | main_cnt_ratio-500__screen_width-ker__uid4-Regist_date-ProductCD




 99%|█████████▉| 151/152 [13:12<00:03,  3.79s/it]

(590540,) | main_cnt_ratio-500__version_id_30-ker__uid4-Regist_date-ProductCD
(506691,) | main_cnt_ratio-500__version_id_30-ker__uid4-Regist_date-ProductCD




100%|██████████| 152/152 [13:17<00:00,  4.00s/it]



In [ ]:
def _calc_agg_category_func(df, key, value):
    new_fe_name_sample = f'{value}_groupby_{"_".join(key)}_n_sample'
    new_fe_name_ratio = f'{value}_groupby_{"_".join(key)}_ratio'
    tmp = df.groupby(key+[value]).size().rename(new_fe_name_sample).reset_index()
    tmp2 = tmp.groupby(key).size().rename("n_total").reset_index()
    result = pd.merge(tmp, tmp2, on=key, how="left")
    result[new_fe_name_ratio] = result["n_total"] / result[new_fe_name_sample]
    # concat
    df = pd.merge(df, result, on=key+[value], how="left")
    return df[[new_fe_name_sample, new_fe_name_ratio]]

In [ ]:
list_base_key = cols_uid
cat_list = cols_M
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

In [23]:
list_base_key = cols_uid
cat_list = cols_domain
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | cnt_ratio-P_emaildomain-ker__uid2-Regist_date
(506691,) | cnt_ratio-P_emaildomain-ker__uid2-Regist_date
(590540,) | cnt_ratio-R_emaildomain-ker__uid2-Regist_date
(506691,) | cnt_ratio-R_emaildomain-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__P_emaildomain_prefix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__P_emaildomain_prefix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__P_emaildomain_suffix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__P_emaildomain_suffix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__R_emaildomain_prefix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__R_emaildomain_prefix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__R_emaildomain_suffix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__R_emaildomain_suffix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__cnt_P_emaildomain-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__cnt_P_emaildomain-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__cnt_R_emaildomain-ker__uid2-Regist_d

In [24]:
list_base_key = cols_uid
cat_list = cols_time
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | cnt_ratio-hour-ker__uid2-Regist_date
(506691,) | cnt_ratio-hour-ker__uid2-Regist_date
(590540,) | cnt_ratio-time_zone-ker__uid2-Regist_date
(506691,) | cnt_ratio-time_zone-ker__uid2-Regist_date
(590540,) | cnt_ratio-hour-ker__uid3-Regist_date
(506691,) | cnt_ratio-hour-ker__uid3-Regist_date
(590540,) | cnt_ratio-time_zone-ker__uid3-Regist_date
(506691,) | cnt_ratio-time_zone-ker__uid3-Regist_date
(590540,) | cnt_ratio-hour-ker__uid4-Regist_date
(506691,) | cnt_ratio-hour-ker__uid4-Regist_date
(590540,) | cnt_ratio-time_zone-ker__uid4-Regist_date
(506691,) | cnt_ratio-time_zone-ker__uid4-Regist_date


In [ ]:
list_base_key = cols_uid
cat_list = cols_501
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)